In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import pickle
import sklearn
from sklearn import metrics

from sklearn.manifold import TSNE
from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, accuracy_score, recall_score, roc_auc_score

RandomSeed = 42
np.random.seed(RandomSeed)

sns.set(font_scale = 1.4)
sns.set_style("white")
sns.set_style("ticks", {"xtick.major.size": 6, "ytick.major.size": 0})

pd.set_option("display.max_colwidth", False)
pd.set_option('display.expand_frame_repr', False)
sns.set(font_scale = 1.2)


In [2]:
def train_model(df_training, training_categories, num_trials, model_name, clf):
    df_training = df_training[df_training['Category'].isin(training_categories)]
    df_training['Category'] = df_training['Category'].map({str(training_categories[0]): 1, training_categories[1]: 0})
    df_training = df_training.reset_index(drop=True)
    df_training = df_training.reindex(sorted(df_training.columns), axis=1)

    gss = GroupShuffleSplit(n_splits = num_trials, test_size = 0.1, random_state=42)
    
    y = df_training['Category']
    groups = df_training['Uniprot_ID']
    X = df_training.drop(columns={'Category', 'Uniprot_ID', 'Sequence'})
    
    accuracy = []; precision = []
    recall = []; roc_auc = []
    for train_index, test_index in gss.split(X, y, groups=groups):
        clf.fit(X.loc[train_index], y.loc[train_index])
        score = clf.score(X.loc[test_index], y.loc[test_index])
        accuracy.append(accuracy_score(clf.predict(X.loc[test_index]), y.loc[test_index]))
        precision.append(precision_score(clf.predict(X.loc[test_index]), y.loc[test_index]))
        recall.append(recall_score(clf.predict(X.loc[test_index]), y.loc[test_index]))
        roc_auc.append(roc_auc_score(y.loc[test_index], clf.predict_proba(X.loc[test_index])[:,1]))
    print('Random forest model: accuracy =' + str(sum(accuracy)/num_trials) + '+/-' + str(np.std(accuracy)/np.sqrt(num_trials)))
    print('Random forest model: precision =' + str(sum(precision)/num_trials) + '+/-' + str(np.std(precision)/np.sqrt(num_trials)))
    print('Random forest model: recall =' + str(sum(recall)/num_trials) + '+/-' + str(np.std(recall)/np.sqrt(num_trials)))
    print('Random forest model: ROC-AUC =' + str(sum(roc_auc)/num_trials) + '+/-' + str(np.std(roc_auc)/np.sqrt(num_trials)))
    
    clf.fit(X, y)
    pickle.dump(clf, open('Models/' + str(model_name) + '.sav', 'wb'))
    

    
def predict(model_name, df_predictions, data_set_name):
    model = pickle.load(open('Models/' + str(model_name) + '.sav', 'rb'))
    df_predictions = df_predictions.reset_index(drop=True)
    df_predictions = df_predictions.reindex(sorted(df_predictions.columns), axis=1)
    
    trial = df_predictions.drop(['Uniprot_ID', 'Category', 'Sequence'], axis=1).to_numpy()
    df_predictions['prediction'] = model.predict_proba(trial)[:,1]
    df_predictions = df_predictions.rename(columns={"prediction": "prediction_" + str(model_name)})
    df_predictions.to_csv('Predictions/' + str(data_set_name) + '/predictions_' + str(model_name) + '.csv')

In [3]:
def train_multicat_model(df_training, num_trials, model_name, clf):
    df_training['Category'] = df_training['Category'].map({'LLPS+': 0, 'LLPS-': 1, 'PDB*': 2})
    df_training = df_training.reset_index(drop = True)
    df_training = df_training.reindex(sorted(df_training.columns), axis=1)

    y = df_training['Category']
    groups = df_training['Uniprot_ID']
    X = df_training.drop(columns={'Category', 'Uniprot_ID', 'Sequence'})
    clf.fit(X, y)
    pickle.dump(clf, open('Models/' + str(model_name) + '.sav', 'wb'))

def predict_multiclass(model_name, df_predictions, data_set_name):
    model = pickle.load(open('Models/' + str(model_name) + '.sav', 'rb'))
    df_predictions = df_predictions.reset_index(drop=True)
    df_predictions = df_predictions.reindex(sorted(df_predictions.columns), axis=1)

    trial = df_predictions.drop(['Uniprot_ID', 'Category', 'Sequence'], axis=1).to_numpy()
    df_predictions['prediction'] = model.predict_proba(trial)[:,0] + 0.5* model.predict_proba(trial)[:,1]
    df_predictions = df_predictions.rename(columns={"prediction": "prediction_" + str(model_name)})
    df_predictions.to_csv('Predictions/' + str(data_set_name) + '/predictions_' + str(model_name) +  '.csv')


# Set up the data

In [4]:
combined = pd.read_csv('Features/training_data_features.csv')
swissprot_all = pd.read_csv('Features/swissprot_features.csv')
overlapping = swissprot_all[swissprot_all['Uniprot_ID'].isin(combined['Uniprot_ID'])]
swissprot_train_removed = swissprot_all[swissprot_all['Uniprot_ID'].isin(overlapping['Uniprot_ID']) == False]

ext_pos = pd.read_csv('Features/external_data_pos_features.csv')
ext_neg = pd.read_csv('Features/external_data_neg_features.csv')
ext_pos['Category'] = 'ext_pos'
ext_neg['Category'] = 'ext_neg'


data_all = pd.concat([swissprot_train_removed, combined, ext_pos, ext_neg], axis = 0)
data_all = data_all[data_all['Sequence_length'] > 6]
data_all = data_all[data_all['Sequence_length'] < 6000]
data_all = data_all.drop(columns = ['LCR_sequence', 'reshuffled_sequence', 'delta_5_HB_RS'])

w2v_cols = data_all.columns[[col.isdigit() for col in data_all.columns]]
data_w2v = data_all[w2v_cols]
info_cols = ['Uniprot_ID', 'Sequence', 'Category']
data_w2v = pd.concat([data_w2v, data_all[info_cols]], axis = 1)

phys_feature_cols_temp = list(np.setdiff1d(list(data_all.columns), info_cols))
phys_feature_cols = list(np.setdiff1d(phys_feature_cols_temp, w2v_cols))
data_phys = data_all[phys_feature_cols]
data_info = data_all[info_cols]
data_phys = pd.concat([data_phys, data_all[info_cols]], axis = 1)

# Training & predictions on Swiss-Prot

In [27]:
#1. Physical features
data_phys_sel = data_phys[{'Category', 'Uniprot_ID',
         'Hydrophobicity', 'Shannon_entropy', 'LCR_frac', 'IDR_frac',
         'Arom_frac', 'Cation_frac', 'Sequence',
         }]

clf_phys = RandomForestClassifier(n_estimators = 100, random_state=42)


#Binary models
n = 25
training_categories = ['LLPS+', 'PDB*']
training_data = data_phys_sel[data_phys_sel['Category'].isin(['LLPS+', 'LLPS-', 'PDB*'])]
phys_features_pred_1 = train_model(training_data, training_categories, num_trials = n, model_name = 'phys_1', clf = clf_phys)

predict_on_SP = data_phys_sel[data_phys_sel['Category'] == 'Swiss-Prot']
predict_on_ext_pos = data_phys_sel[data_phys_sel['Category'] == 'ext_pos']
predict_on_ext_neg = data_phys_sel[data_phys_sel['Category'] == 'ext_neg']
predict('phys_1', predict_on_SP, 'Swissprot')
predict('phys_1', predict_on_ext_neg, 'ext_neg')
predict('phys_1', predict_on_ext_pos, 'ext_pos')

training_categories = ['LLPS+', 'LLPS-']
training_data = data_phys_sel[data_phys_sel['Category'].isin(['LLPS+', 'LLPS-', 'PDB*'])]
phys_features_pred_2 = train_model(training_data, training_categories, num_trials = n, model_name = 'phys_2', clf = clf_phys)

predict_on_SP = data_phys_sel[data_phys_sel['Category'] == 'Swiss-Prot']
predict_on_ext_pos = data_phys_sel[data_phys_sel['Category'] == 'ext_pos']
predict_on_ext_neg = data_phys_sel[data_phys_sel['Category'] == 'ext_neg']
predict('phys_2', predict_on_SP, 'Swissprot')
predict('phys_2', predict_on_ext_neg, 'ext_neg')
predict('phys_2', predict_on_ext_pos, 'ext_pos')

training_categories = ['PDB*', 'LLPS-']
training_data = data_phys_sel[data_phys_sel['Category'].isin(['LLPS+', 'LLPS-', 'PDB*'])]
phys_features_pred_3 = train_model(training_data, training_categories, num_trials = n, model_name = 'phys_3', clf = clf_phys)

predict_on_SP = data_phys_sel[data_phys_sel['Category'] == 'Swiss-Prot']
predict_on_ext_pos = data_phys_sel[data_phys_sel['Category'] == 'ext_pos']
predict_on_ext_neg = data_phys_sel[data_phys_sel['Category'] == 'ext_neg']
predict('phys_3', predict_on_SP, 'Swissprot')
predict('phys_3', predict_on_ext_neg, 'ext_neg')
predict('phys_3', predict_on_ext_pos, 'ext_pos')


#Multiclass model
clf_phys = RandomForestClassifier(n_estimators = 50, max_depth = 5, random_state = 42)
training_data = data_phys_sel[data_phys_sel['Category'].isin(['LLPS+', 'LLPS-', 'PDB*'])]
phys_features_pred_multi = train_multicat_model(training_data, num_trials = 25, model_name = 'phys_multi', clf = clf_phys)

predict_on_SP = data_phys_sel[data_phys_sel['Category'] == 'Swiss-Prot']
predict_on_ext_pos = data_phys_sel[data_phys_sel['Category'] == 'ext_pos']
predict_on_ext_neg = data_phys_sel[data_phys_sel['Category'] == 'ext_neg']
predict_multiclass('phys_multi', predict_on_SP, 'Swissprot')
predict_multiclass('phys_multi', predict_on_ext_neg, 'ext_neg')
predict_multiclass('phys_multi', predict_on_ext_pos, 'ext_pos')

<ipython-input-2-642d93db321c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['Category'] = df_training['Category'].map({str(training_categories[0]): 1, training_categories[1]: 0})


Random forest model: accuracy =0.9381384102975026+/-0.007935073272259969
Random forest model: precision =0.9221392374395471+/-0.014262076975556648
Random forest model: recall =0.9453181587400655+/-0.013138980734518826
Random forest model: ROC-AUC =0.9678019999092754+/-0.007451111892813176


<ipython-input-2-642d93db321c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['Category'] = df_training['Category'].map({str(training_categories[0]): 1, training_categories[1]: 0})


Random forest model: accuracy =0.6433872055064213+/-0.01618602072047163
Random forest model: precision =0.7368189592594743+/-0.025700170764370554
Random forest model: recall =0.6857878891740486+/-0.024105425103566315
Random forest model: ROC-AUC =0.6939131172672168+/-0.018119491139988834


<ipython-input-2-642d93db321c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['Category'] = df_training['Category'].map({str(training_categories[0]): 1, training_categories[1]: 0})


Random forest model: accuracy =0.8906718087603118+/-0.014146195346163229
Random forest model: precision =0.9534256919551037+/-0.011889409669450656
Random forest model: recall =0.8788694606047548+/-0.017412622168814878
Random forest model: ROC-AUC =0.9039794486436459+/-0.02233283555040995


<ipython-input-3-438d43cfe171>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['Category'] = df_training['Category'].map({'LLPS+': 0, 'LLPS-': 1, 'PDB*': 2})


In [31]:
#2. Models with w2v embeddings:
clf_w2v = RandomForestClassifier(n_estimators = 100, max_depth = 5, random_state = 42)

#Binary models
n = 25
training_categories = ['LLPS+', 'PDB*']
training_data = data_w2v[data_w2v['Category'].isin(['LLPS+', 'LLPS-', 'PDB*'])]
w2v_pred_1 = train_model(training_data, training_categories, num_trials = n, model_name = 'w2v_1', clf = clf_w2v)

predict_on_SP = data_w2v[data_w2v['Category'] == 'Swiss-Prot']
predict_on_ext_pos = data_w2v[data_w2v['Category'] == 'ext_pos']
predict_on_ext_neg = data_w2v[data_w2v['Category'] == 'ext_neg']
predict('w2v_1', predict_on_SP, 'Swissprot')
predict('w2v_1', predict_on_ext_neg, 'ext_neg')
predict('w2v_1', predict_on_ext_pos, 'ext_pos')

training_categories = ['LLPS+', 'LLPS-']
training_data = data_w2v[data_w2v['Category'].isin(['LLPS+', 'LLPS-', 'PDB*'])]
w2v_pred_2 = train_model(training_data, training_categories, num_trials = n, model_name = 'w2v_2', clf = clf_w2v)
predict_on_SP = data_w2v[data_w2v['Category'] == 'Swiss-Prot']
predict_on_ext_pos = data_w2v[data_w2v['Category'] == 'ext_pos']
predict_on_ext_neg = data_w2v[data_w2v['Category'] == 'ext_neg']
predict('w2v_2', predict_on_SP, 'Swissprot')
predict('w2v_2', predict_on_ext_neg, 'ext_neg')
predict('w2v_2', predict_on_ext_pos, 'ext_pos')

training_categories = ['PDB*', 'LLPS-']
training_data = data_w2v[data_w2v['Category'].isin(['LLPS+', 'LLPS-', 'PDB*'])]
w2v_pred_3 = train_model(training_data, training_categories, num_trials = n, model_name = 'w2v_3', clf = clf_w2v)
predict_on_SP = data_w2v[data_w2v['Category'] == 'Swiss-Prot']
predict_on_ext_pos = data_w2v[data_w2v['Category'] == 'ext_pos']
predict_on_ext_neg = data_w2v[data_w2v['Category'] == 'ext_neg']
predict('w2v_3', predict_on_SP, 'Swissprot')
predict('w2v_3', predict_on_ext_neg, 'ext_neg')
predict('w2v_3', predict_on_ext_pos, 'ext_pos')

#Multiclass model
training_data = data_w2v[data_w2v['Category'].isin(['LLPS+', 'LLPS-', 'PDB*'])]
w2v_pred_multi = train_multicat_model(training_data, num_trials = 25, model_name = 'w2v_multi', clf = clf_w2v)
predict_on_SP = data_w2v[data_w2v['Category'] == 'Swiss-Prot']
predict_on_ext_pos = data_w2v[data_w2v['Category'] == 'ext_pos']
predict_on_ext_neg = data_w2v[data_w2v['Category'] == 'ext_neg']
predict_multiclass('w2v_multi', predict_on_SP, 'Swissprot')
predict_multiclass('w2v_multi', predict_on_ext_neg, 'ext_neg')
predict_multiclass('w2v_multi', predict_on_ext_pos, 'ext_pos')


<ipython-input-2-642d93db321c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['Category'] = df_training['Category'].map({str(training_categories[0]): 1, training_categories[1]: 0})


Random forest model: accuracy =0.912546500501352+/-0.011239810713163993
Random forest model: precision =0.842050184650804+/-0.027644120510572513
Random forest model: recall =0.9705757575757576+/-0.009619519706620932
Random forest model: ROC-AUC =0.9825583676958289+/-0.004332965944618376


<ipython-input-2-642d93db321c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['Category'] = df_training['Category'].map({str(training_categories[0]): 1, training_categories[1]: 0})


Random forest model: accuracy =0.6609274491914741+/-0.02024918277490066
Random forest model: precision =0.749958105108383+/-0.03153301855452162
Random forest model: recall =0.7035283409401057+/-0.024532376212781547
Random forest model: ROC-AUC =0.7164512290336532+/-0.01928082232416829


<ipython-input-2-642d93db321c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['Category'] = df_training['Category'].map({str(training_categories[0]): 1, training_categories[1]: 0})


Random forest model: accuracy =0.876543442701069+/-0.016080770136648152
Random forest model: precision =0.9819694031458739+/-0.006469361176672539
Random forest model: recall =0.8448485674971744+/-0.02323571406787151
Random forest model: ROC-AUC =0.9161270437017894+/-0.01545078132500868


<ipython-input-3-438d43cfe171>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['Category'] = df_training['Category'].map({'LLPS+': 0, 'LLPS-': 1, 'PDB*': 2})


# Combine predictions

In [32]:
category = 'Swissprot'
phys_pred_1 = pd.read_csv('Predictions/' + str(category) + '/predictions_phys_1.csv')
phys_pred_2 = pd.read_csv('Predictions/' + str(category) +'/predictions_phys_2.csv')
phys_pred_multi = pd.read_csv('Predictions/' + str(category) + '/predictions_phys_multi.csv')

w2v_pred_1 = pd.read_csv('Predictions/' + str(category) + '/predictions_w2v_1.csv')
w2v_pred_2 = pd.read_csv('Predictions/' + str(category) + '/predictions_w2v_2.csv')
w2v_pred_multi = pd.read_csv('Predictions/' + str(category) + '/predictions_w2v_multi.csv')


predictions = pd.concat([phys_pred_1[{'Category', 'Uniprot_ID', 'Sequence', 'prediction_phys_1'}],
                phys_pred_2[{'prediction_phys_2'}],
                phys_pred_multi[{'prediction_phys_multi'}],
                w2v_pred_1[{'prediction_w2v_1'}],
                w2v_pred_2[{'prediction_w2v_2'}],
                w2v_pred_multi[{'prediction_w2v_multi'}]], axis = 1)

predictions.to_csv('Predictions/' + str(category) + '/all_predictions.csv', index=False)

In [33]:
category = 'ext_pos'
phys_pred_1 = pd.read_csv('Predictions/' + str(category) + '/predictions_phys_1.csv')
phys_pred_2 = pd.read_csv('Predictions/' + str(category) +'/predictions_phys_2.csv')
phys_pred_multi = pd.read_csv('Predictions/' + str(category) + '/predictions_phys_multi.csv')

w2v_pred_1 = pd.read_csv('Predictions/' + str(category) + '/predictions_w2v_1.csv')
w2v_pred_2 = pd.read_csv('Predictions/' + str(category) + '/predictions_w2v_2.csv')
w2v_pred_multi = pd.read_csv('Predictions/' + str(category) + '/predictions_w2v_multi.csv')


predictions = pd.concat([phys_pred_1[{'Category', 'Uniprot_ID', 'Sequence', 'prediction_phys_1'}],
                phys_pred_2[{'prediction_phys_2'}],
                phys_pred_multi[{'prediction_phys_multi'}],
                w2v_pred_1[{'prediction_w2v_1'}],
                w2v_pred_2[{'prediction_w2v_2'}],
                w2v_pred_multi[{'prediction_w2v_multi'}]], axis = 1)

predictions.to_csv('Predictions/' + str(category) + '/all_predictions.csv', index=False)

In [34]:
category = 'ext_neg'
phys_pred_1 = pd.read_csv('Predictions/' + str(category) + '/predictions_phys_1.csv')
phys_pred_2 = pd.read_csv('Predictions/' + str(category) +'/predictions_phys_2.csv')
phys_pred_multi = pd.read_csv('Predictions/' + str(category) + '/predictions_phys_multi.csv')

w2v_pred_1 = pd.read_csv('Predictions/' + str(category) + '/predictions_w2v_1.csv')
w2v_pred_2 = pd.read_csv('Predictions/' + str(category) + '/predictions_w2v_2.csv')
w2v_pred_multi = pd.read_csv('Predictions/' + str(category) + '/predictions_w2v_multi.csv')

predictions = pd.concat([phys_pred_1[{'Category', 'Uniprot_ID', 'Sequence', 'prediction_phys_1'}],
                phys_pred_2[{'prediction_phys_2'}],
                phys_pred_multi[{'prediction_phys_multi'}],
                w2v_pred_1[{'prediction_w2v_1'}],
                w2v_pred_2[{'prediction_w2v_2'}],
                w2v_pred_multi[{'prediction_w2v_multi'}]], axis = 1)

predictions.to_csv('Predictions/' + str(category) + '/all_predictions.csv', index=False)